# Crack detection strategy

The detection algorithm uses the knowledge about shear crack propagation 
 - distance between cracks is related to the shielded length given by the reinforcement ratio      and bond properties
 - cracks start from the bottom of the cross section
 - cracks propagate upwards and bend in the direction of an increasing cross-sectional moment
 
The crack pattern in the shear zone gets most visible prior shortly before the maximum load has been reached. Near to the failure, the regularity of the crack pattern is lost and the specimen cannot be viewed as a series of individual shear cracks any more. The initiation of failure via a horizontal crossing crack initiated by a vertical stress component is initiated either near the reinforcement layer due to the dowel action or by a crack tip propagating horizontally through a compression zone.

# Full crack detection strategy

This observation is exploited by the crack detection algorithm by adapting the following strategy. 

 - The displacement field $U_{IJa}^{F_\gamma}$ corresponding to $F_\gamma = 0.9 F_\max$ is taken first to calculate the strain, damage and stress fields.
 - The DIC-FE state fields are interpolated and smoothed as a basis for the crack detection step. The crack detection algorithm is employed to detect the crack path $\mathcal{C}(F_\gamma)$ at the near-failure level of load. The algorithm delivers the crack tip positions.
 - The obtained crack path is then refined by analyzing the cracking process in reverse order $F = [F_\gamma, 0]$. The difference between damage fields $\omega_{MN}(F_\gamma)$ and $\omega_{MN}(F), n = [0, n_C-1]$  obtained using DIC-FE-MDM calculation for each $U_{IJa}$ state are analyzed to refine the crack geometry in the previous states of loading. After that, the load range between the crack detection state and the ultimate load is analyzed based on the known cracking state at the level $F\tilde{}$.
 
# Current strategy

The crack detection load level $\gamma$ is used to identify the primary cracks. The cracks that develop at the level $\gamma = 0.9$ up to the 0.4 of the cross sectional height are recognized as primary cracks. Partially, secondary cracks and noise in DIC data and FE approximation are avoided  by smoothing the damage field within a radius that reflects the element resolution / characteristic length? Currently, the primary cracks are identified first to obtain the crack zones.

 - Add the 

In [1]:
%matplotlib widget
import ibvpy.api as ib
import matplotlib.pylab as plt
from matplotlib import cm
from scipy.interpolate import interp2d
from scipy.signal import argrelextrema
from bmcs_shear.api import CrackBridgeAdv
from bmcs_shear.dic_crack import\
    DICGrid, DICStateFields, DICCrackList, DICCrack, DICCrackCOR
from bmcs_shear.shear_crack.crack_path import get_T_Lab
import numpy as np
np.seterr(divide='ignore', invalid='ignore');

In [2]:
dic_grid = DICGrid(start_t=0, end_t=1, U_factor=100, dir_name='B10_TV1')

In [3]:
dsf = DICStateFields(dic_grid=dic_grid, t=0.9)
dsf.tmodel_.trait_set(E=5000, c_T=0, nu=0.18, epsilon_0=0.0005, epsilon_f=0.01);
dsf.eval()

In [4]:
dcl = DICCrackList(dsf=dsf)

In [5]:
sp = dcl.items[3]

In [6]:
sp.X_neutral_a

array([206.66073323, 214.74792509])

In [7]:
x_00 = sp.dic_crack.C_cubic_spline(sp.z_N)

In [8]:
x_00

array([256.79270438])

In [9]:
sp.M

21331292.063399736

In [10]:
dcl.interact()

In [6]:
crack_0 = dcl.items[2]

In [9]:
fig, ax = plt.subplots(1,1)
crack_0.dic_crack.plot_u_Nib(ax)
crack_0.dic_crack.plot_x_Na(ax)
crack_0.plot_COR(ax)
crack_0.plot_cor_markers(ax)
ax.axis('equal')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(248.36770344360858, 480.1367773488707, -7.550799748321647, 274.79547068760746)

In [ ]:
crack_0.M_N[20]

In [ ]:
dc_cor = DICCrackCOR(dic_crack=dcl.items[2])

In [ ]:
dc_cor.M_N.dtype

In [ ]:
slice_M_N = slice(dc_cor.M_N,dc_cor.M_N+3)

In [ ]:
X_mNa = np.array([dc_cor.X_MNa[dc_cor.M_N+i, np.arange(len(dc_cor.M_N))] for i in range(5)])

In [ ]:
X_mNa[-1,0]

In [ ]:
dc_cor.M0

In [ ]:
dc_cor.interact()

In [ ]:
dc_cor.X0_a - dc_cor.U0_a

In [19]:
dc_cor.x_ref_MNa_scaled[dc_cor.M0, :dc_cor.N0_max]

array([[330.87272231,   4.85905338],
       [330.87254552,  13.86303457],
       [330.87358701,  22.868741  ],
       [330.87234085,  31.88218863],
       [330.87220829,  40.89703656]])